In [1]:
import pandas as pd
import itertools
import requests

In [2]:
df_raw_imdb = pd.read_csv('raw_imdb.csv')
df_raw_imdb = df_raw_imdb[['primaryTitle', 'startYear', 'numVotes', 'averageRating', 'genres']]
df_raw_imdb

,primaryTitle,startYear,numVotes,averageRating,genres
0,Dante's Inferno,1911,3854,7.0,"Adventure,Drama,Fantasy"
1,Fantômas: In the Shadow of the Guillotine,1913,2646,6.9,"Crime,Drama"
2,Ingeborg Holm,1913,1542,7.0,Drama
3,Fantomas: The Man in Black,1913,1796,6.9,"Crime,Drama"
4,Cabiria,1914,4190,7.1,"Adventure,Drama,History"
...,...,...,...,...,...
17219,Mogul Mowgli,2020,3373,6.6,"Drama,Music"
17220,Min pappa Marianne,2020,2421,6.8,"Comedy,Drama"
17221,Kaithi,2019,47492,8.4,"Action,Crime,Thriller"
17222,Herself,2020,5163,7.0,Drama


In [3]:
genres_list = df_raw_imdb['genres'].dropna().str.split(',')
genres_set = set(itertools.chain.from_iterable(genres_list))
genres_set.add('')
genres_set

{'',
 'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western'}

In [4]:
len(genres_set)

25

In [5]:
def get_tmdb_info(title, year):
    api_key = '7d50f2eef53ed745dbc5a731c72dfa36'
    search_url = f'https://api.themoviedb.org/3/search/movie'
    params = {
        'api_key': api_key,
        'query': title,
        'year': year,
        'language': 'zh-TW'
    }
    resp = requests.get(search_url, params=params).json()
    if resp['results']:
        movie_id = resp['results'][0]['id']
        # 取得中文標題
        taiwan_title = resp['results'][0].get('title', '')
        # 取得預告片
        video_url = f'https://api.themoviedb.org/3/movie/{movie_id}/videos'
        video_resp = requests.get(video_url, params={'api_key': api_key}).json()
        trailer_link = ''
        poster_path = resp['results'][0].get('poster_path', '')
        poster_path = f"https://image.tmdb.org/t/p/w500{poster_path}"
        for v in video_resp.get('results', []):
            if v['type'] == 'Trailer' and v['site'] == 'YouTube':
                trailer_link = f"https://www.youtube.com/watch?v={v['key']}"
                break
        return taiwan_title, trailer_link, poster_path
    return '', '', ''

In [6]:
for genre in genres_set:
    if genre == '':
        df_genre = df_raw_imdb[df_raw_imdb['genres'].isnull()]
        filename = 'empty.csv'
    else:
        df_genre = df_raw_imdb[df_raw_imdb['genres'].fillna('').str.contains(genre)]
        filename = f'{genre}.csv'
    percentile_60 = df_genre['numVotes'].quantile(0.6)
    threshold = min(percentile_60, 10000)
    df_genre_filtered = df_genre[df_genre['numVotes'] > threshold]
    print(f'Processing genre: {genre}')
    print(f'numVotes threshold: {threshold}')
    print(len(df_genre_filtered))
    df_genre_filtered.drop(columns=['genres'], inplace=True)
    df_genre_filtered = df_genre_filtered[df_genre_filtered["averageRating"]>=6.5]
    df_genre_filtered.to_csv(filename, index=False)

Processing genre: 
numVotes threshold: 3106.0
4
Processing genre: Thriller
numVotes threshold: 10000
885
Processing genre: Animation
numVotes threshold: 10000
378
Processing genre: Horror
numVotes threshold: 10000
400
Processing genre: Mystery
numVotes threshold: 10000
587
Processing genre: Film-Noir
numVotes threshold: 5444.599999999999
117
Processing genre: Sport
numVotes threshold: 9029.399999999996
168
Processing genre: Documentary
numVotes threshold: 4101.0
598
Processing genre: Musical
numVotes threshold: 6582.4
153
Processing genre: Adult
numVotes threshold: 2073.0
0
Processing genre: Biography
numVotes threshold: 10000
683
Processing genre: Music
numVotes threshold: 6913.8
434
Processing genre: Adventure
numVotes threshold: 10000
983
Processing genre: Crime
numVotes threshold: 10000
1396


C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_filtered.drop(columns=['genres'], inplace=True)
C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_filtered.drop(columns=['genres'], inplace=True)
C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

Processing genre: Sci-Fi
numVotes threshold: 10000
311
Processing genre: Action
numVotes threshold: 10000
1274
Processing genre: Drama
numVotes threshold: 9568.2
4677
Processing genre: Western
numVotes threshold: 10000
102
Processing genre: Romance
numVotes threshold: 8344.999999999998
1255
Processing genre: Family
numVotes threshold: 9073.400000000001
258
Processing genre: News
numVotes threshold: 4710.8
9
Processing genre: Comedy
numVotes threshold: 9750.2
2064
Processing genre: History
numVotes threshold: 8174.799999999999
405
Processing genre: War
numVotes threshold: 8103.8
266


C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_filtered.drop(columns=['genres'], inplace=True)
C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_filtered.drop(columns=['genres'], inplace=True)
C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

Processing genre: Fantasy
numVotes threshold: 10000
343


C:\Users\f8210\AppData\Local\Temp\ipykernel_6736\1627860659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_filtered.drop(columns=['genres'], inplace=True)


In [ ]:
with pd.ExcelWriter('all_genres.xlsx', engine='openpyxl') as writer:
    for genre in genres_set:
        if genre == '':
            genre_name = 'Empty'
            df_genre = pd.read_csv('empty.csv')
        else:
            genre_name = genre
            
            df_genre = pd.read_csv(f'{genre}.csv')
        
        percentile_90 = df_genre['averageRating'].quantile(0.9)
        print(f'Processing genre: {genre_name}')
        
        df_genre['taiwanTitle'] = ''
        df_genre['trailerLink'] = ''
        df_genre['posterPath'] = ''
        for idx, row in df_genre.iterrows():
            if row['averageRating'] >= percentile_90:
                tw_title, trailer, poster_path = get_tmdb_info(row['primaryTitle'], row['startYear'])
                df_genre.at[idx, 'taiwanTitle'] = tw_title
                df_genre.at[idx, 'trailerLink'] = trailer    
                df_genre.at[idx, 'posterPath'] = poster_path    
                print(f"Processed {row['primaryTitle']} ({row['startYear']}): {tw_title}, {trailer}")
        
        # 存到 Excel 的不同 sheet
        df_genre.to_excel(writer, sheet_name=genre_name[:31], index=False)

In [2]:
# 讀取 Excel 檔案的所有 sheet 名稱
excel_path = "all_genres.xlsx"
sheets_dict = pd.read_excel(excel_path, sheet_name=None)  # 讀取所有 sheets

# 指定要排序的欄位名稱
sort_column = "averageRating"

# 建立一個新的 dict 來存放排序後的 DataFrame
sorted_sheets = {}

for sheet_name, df in sheets_dict.items():
    if sort_column in df.columns:
        sorted_df = df.sort_values(by=sort_column, ascending=False)
        sorted_sheets[sheet_name] = sorted_df
    else:
        print(f"Sheet '{sheet_name}' 中找不到欄位 '{sort_column}'，略過")

# 如果你想把結果寫回新的 Excel 檔案
with pd.ExcelWriter("movie recommendation.xlsx", engine='openpyxl') as writer:
    for sheet_name, sorted_df in sorted_sheets.items():
        sorted_df.to_excel(writer, sheet_name=sheet_name, index=False)
